# Train a Model with SageMaker AutoPilot
We will use AutoPilot to predict sentiment of customer reviews.

# Introduction

Amazon SageMaker Autopilot is a service to perform automated machine learning (AutoML) on your datasets.  AutoPilot is available through the UI or AWS SDK.  In this notebook, we will use the AWS SDK to create and deploy a text processing and sentiment classification machine learning pipeline.

# Setup

Let's start by specifying:

* The S3 bucket and prefix to use to train our model.  _Note:  This should be in the same region as this notebook._
* The IAM role of this notebook needs access to your data.

In [1]:
import boto3
import sagemaker
import pandas as pd

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name='sagemaker', region_name=region)

# Dataset

In [2]:
%store -r header_train_s3_uri

print(header_train_s3_uri)

s3://sagemaker-us-east-1-835319576252/data/amazon_reviews_us_Digital_Software_v1_00_header.csv


In [3]:
!aws s3 ls $header_train_s3_uri

2020-05-16 05:20:29   13660674 amazon_reviews_us_Digital_Software_v1_00_header.csv


# Setup the S3 Location for the AutoPilot-Generated Assets 
This include Jupyter Notebooks (Analysis), Python Scripts (Feature Engineering), and Trained Models.

In [4]:
prefix_model_output = 'models/autopilot'

model_output_s3_uri = 's3://{}/{}'.format(bucket, prefix_model_output)

print(model_output_s3_uri)


s3://sagemaker-us-east-1-835319576252/models/autopilot


In [5]:
max_candidates = 3

job_config = {
    'CompletionCriteria': {
      'MaxRuntimePerTrainingJobInSeconds': 600,
      'MaxCandidates': max_candidates,
      'MaxAutoMLJobRuntimeInSeconds': 3600
    },
}

input_data_config = [{
      'DataSource': {
        'S3DataSource': {
          'S3DataType': 'S3Prefix',
          'S3Uri': '{}'.format(header_train_s3_uri)
        }
      },
      'TargetAttributeName': 'star_rating'
    }
]

output_data_config = {
    'S3OutputPath': '{}'.format(model_output_s3_uri)
}

# Launch the SageMaker AutoPilot job

We can now launch the job by calling the `create_auto_ml_job` API.

In [6]:
from time import gmtime, strftime, sleep
timestamp_suffix = strftime('%d-%H-%M-%S', gmtime())

auto_ml_job_name = 'automl-dm-' + timestamp_suffix
print('AutoMLJobName: ' + auto_ml_job_name)

AutoMLJobName: automl-dm-16-05-20-45


_Note that we are not specifying the `ProblemType`.  AutoPilot will automatically detect if we're using regression or classification (binary or multi-class)._

In [7]:
sm.create_auto_ml_job(AutoMLJobName=auto_ml_job_name,
                      InputDataConfig=input_data_config,
                      OutputDataConfig=output_data_config,
                      AutoMLJobConfig=job_config,
#                      ProblemType="Classification",
                      RoleArn=role)

{'AutoMLJobArn': 'arn:aws:sagemaker:us-east-1:835319576252:automl-job/automl-dm-16-05-20-45',
 'ResponseMetadata': {'RequestId': '5bec7f59-1475-454c-b9d9-44b134d23d97',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '5bec7f59-1475-454c-b9d9-44b134d23d97',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '92',
   'date': 'Sat, 16 May 2020 05:20:45 GMT'},
  'RetryAttempts': 0}}

# Tracking the progress of the AutoPilot job
SageMaker AutoPilot job consists of the following high-level steps: 
* _Data Analysis_ where the data is summarized and analyzed to determine which feature engineering techniques, hyper-parameters, and models to explore.
* _Feature Engineering_ where the data is scrubbed, balanced, combined, and split into train and validation
* _Model Training and Tuning_ where the top performing features, hyper-parameters, and models are selected and trained.

# Analyzing Data

In [8]:
# Sleep for a bit to ensure the AutoML job above has time to start
import time
time.sleep(3)

job = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
job_status = job['AutoMLJobStatus']
job_sec_status = job['AutoMLJobSecondaryStatus']

if job_status not in ('Stopped', 'Failed'):
    while job_status in ('InProgress') and job_sec_status in ('AnalyzingData'):
        job = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
        job_status = job['AutoMLJobStatus']
        job_sec_status = job['AutoMLJobSecondaryStatus']
        print(job_status, job_sec_status)
        sleep(30)
    print("Data analysis complete")
    
print(job)

InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress FeatureEngineering
Data analysis complete
{'AutoMLJobName': 'automl-dm-16-05-20-45', 'AutoMLJobArn': 'arn:aws:sagemaker:us-east-1:835319576252:automl-job/automl-dm-16-05-20-45', 'InputDataConfig': [{'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix', 'S3Uri': 's3://sagemaker-us-east-1-835319576252/data/amazon_reviews_us_Digital_Software_v1_00_header.csv'}}, 'TargetAttributeName': 'star_rating'}], 'OutputDataConfig': {'S3OutputPath': 's3://sagemaker-us-east-1-835319576252/models/autopilot'}, 'RoleArn': 'arn:aws:iam::835319576252:role/service-role/AmazonSageMaker-ExecutionRole-2

In [9]:
job = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
print(job)

{'AutoMLJobName': 'automl-dm-16-05-20-45', 'AutoMLJobArn': 'arn:aws:sagemaker:us-east-1:835319576252:automl-job/automl-dm-16-05-20-45', 'InputDataConfig': [{'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix', 'S3Uri': 's3://sagemaker-us-east-1-835319576252/data/amazon_reviews_us_Digital_Software_v1_00_header.csv'}}, 'TargetAttributeName': 'star_rating'}], 'OutputDataConfig': {'S3OutputPath': 's3://sagemaker-us-east-1-835319576252/models/autopilot'}, 'RoleArn': 'arn:aws:iam::835319576252:role/service-role/AmazonSageMaker-ExecutionRole-20191006T135881', 'AutoMLJobConfig': {'CompletionCriteria': {'MaxCandidates': 3, 'MaxRuntimePerTrainingJobInSeconds': 600, 'MaxAutoMLJobRuntimeInSeconds': 3600}}, 'CreationTime': datetime.datetime(2020, 5, 16, 5, 20, 45, 812000, tzinfo=tzlocal()), 'LastModifiedTime': datetime.datetime(2020, 5, 16, 5, 29, 1, 906000, tzinfo=tzlocal()), 'AutoMLJobStatus': 'InProgress', 'AutoMLJobSecondaryStatus': 'FeatureEngineering', 'GenerateCandidateDefinitionsOnly':

# Feature Engineering

In [10]:
job = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
job_status = job['AutoMLJobStatus']
job_sec_status = job['AutoMLJobSecondaryStatus']
print(job_status)
print(job_sec_status)
if job_status not in ('Stopped', 'Failed'):
    while job_status in ('InProgress') and job_sec_status in ('FeatureEngineering'):
        job = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
        job_status = job['AutoMLJobStatus']
        job_sec_status = job['AutoMLJobSecondaryStatus']
        print(job_status, job_sec_status)
        sleep(30)
    print("Feature engineering complete")
    
print(job)

InProgress
FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress ModelTuning
Feature engineering complete
{'AutoMLJobName': 'automl-dm-16-05-20-45', 'AutoMLJobArn': 'arn:aws:sagemaker:us-east-1:835319576252:automl-job/automl-dm-16-05-20-45', 'InputDataConfig': [{'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix', 'S3Uri': 's3://sagemaker-us-east-1-835319576252/data/amazon_reviews_us_Digital_Software_v1_00_header.csv'}}, 'TargetAttributeName': 'star_rating'}], 'OutputDataConfig': {'S3OutputPath': 's3://sagemaker-us-east-1-835319576252/models/autopilot'}, 'RoleArn

# Model Training and Tuning

In [11]:
job = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
job_status = job['AutoMLJobStatus']
job_sec_status = job['AutoMLJobSecondaryStatus']
print(job_status)
print(job_sec_status)
if job_status not in ('Stopped', 'Failed'):
    while job_status in ('InProgress') and job_sec_status in ('ModelTuning'):
        job = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
        job_status = job['AutoMLJobStatus']
        job_sec_status = job['AutoMLJobSecondaryStatus']
        print(job_status, job_sec_status)
        sleep(30)
    print("Model tuning complete")
    
print(job)

InProgress
ModelTuning
InProgress ModelTuning
InProgress ModelTuning
InProgress ModelTuning
InProgress ModelTuning
InProgress ModelTuning
InProgress ModelTuning
InProgress ModelTuning
InProgress ModelTuning
InProgress ModelTuning
InProgress ModelTuning
Completed MaxCandidatesReached
Model tuning complete
{'AutoMLJobName': 'automl-dm-16-05-20-45', 'AutoMLJobArn': 'arn:aws:sagemaker:us-east-1:835319576252:automl-job/automl-dm-16-05-20-45', 'InputDataConfig': [{'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix', 'S3Uri': 's3://sagemaker-us-east-1-835319576252/data/amazon_reviews_us_Digital_Software_v1_00_header.csv'}}, 'TargetAttributeName': 'star_rating'}], 'OutputDataConfig': {'S3OutputPath': 's3://sagemaker-us-east-1-835319576252/models/autopilot'}, 'RoleArn': 'arn:aws:iam::835319576252:role/service-role/AmazonSageMaker-ExecutionRole-20191006T135881', 'AutoMLJobConfig': {'CompletionCriteria': {'MaxCandidates': 3, 'MaxRuntimePerTrainingJobInSeconds': 600, 'MaxAutoMLJobRuntimeInSec

# _Please Wait Until ^^ Autopilot ^^ Completes Above_

# View Generated Notebooks
Once data analysis is complete, SageMaker AutoPilot generates two notebooks: 
* Data exploration,
* Candidate definition.

## Copy the Generated Notebooks Locally

In [12]:
generated_resources = job['AutoMLJobArtifacts']['DataExplorationNotebookLocation'].rstrip('notebooks/SageMakerAutopilotDataExplorationNotebook.ipynb')
generated_resources

's3://sagemaker-us-east-1-835319576252/models/autopilot/automl-dm-16-05-20-45/sagemaker-automl-candidates/pr-1-51bf24524e5b494a821d9ac140bd58e00e553de599064e39985788efb4'

In [13]:
!aws s3 cp --recursive $generated_resources .

download: s3://sagemaker-us-east-1-835319576252/models/autopilot/automl-dm-16-05-20-45/sagemaker-automl-candidates/pr-1-51bf24524e5b494a821d9ac140bd58e00e553de599064e39985788efb4/generated_module/MANIFEST.in to generated_module/MANIFEST.in
download: s3://sagemaker-us-east-1-835319576252/models/autopilot/automl-dm-16-05-20-45/sagemaker-automl-candidates/pr-1-51bf24524e5b494a821d9ac140bd58e00e553de599064e39985788efb4/generated_module/README.md to generated_module/README.md
download: s3://sagemaker-us-east-1-835319576252/models/autopilot/automl-dm-16-05-20-45/sagemaker-automl-candidates/pr-1-51bf24524e5b494a821d9ac140bd58e00e553de599064e39985788efb4/generated_module/candidate_data_processors/dpp0.py to generated_module/candidate_data_processors/dpp0.py
download: s3://sagemaker-us-east-1-835319576252/models/autopilot/automl-dm-16-05-20-45/sagemaker-automl-candidates/pr-1-51bf24524e5b494a821d9ac140bd58e00e553de599064e39985788efb4/generated_module/candidate_data_processors/dpp1.py to generat

## In the file view, open the following folders:
```
notebooks/
generated_module/
```

Lots of useful information ^^ in these folders ^^

# Viewing All Candidates
Once model tuning is complete, you can view all the candidates (pipeline evaluations with different hyperparameter combinations) that were explored by AutoML and sort them by their final performance metric.

In [14]:
candidates = sm.list_candidates_for_auto_ml_job(AutoMLJobName=auto_ml_job_name, 
                                                SortBy='FinalObjectiveMetricValue')['Candidates']
for index, candidate in enumerate(candidates):
    print(str(index) + "  " 
        + candidate['CandidateName'] + "  " 
        + str(candidate['FinalAutoMLJobObjectiveMetric']['Value']))

0  tuning-job-1-872297ed5107410fbc-001-f28a0f1e  0.4808900058269501
1  tuning-job-1-872297ed5107410fbc-002-011c5072  0.4405739903450012
2  tuning-job-1-872297ed5107410fbc-003-6b1c2c2a  0.2723749876022339


# Inspect Trials using Experiments API
SageMaker AutoPilot automatically creates a new experiment, and pushes information for each trial. 

In [15]:
from sagemaker.analytics import ExperimentAnalytics, TrainingJobAnalytics

exp = ExperimentAnalytics(
    sagemaker_session=sess, 
    experiment_name=auto_ml_job_name + '-aws-auto-ml-job',
)

df = exp.dataframe()
print(df)

                                  TrialComponentName  \
0  tuning-job-1-872297ed5107410fbc-001-f28a0f1e-a...   
1  tuning-job-1-872297ed5107410fbc-002-011c5072-a...   
2  tuning-job-1-872297ed5107410fbc-003-6b1c2c2a-a...   
3  automl-dm--dpp2-rpb-1-a57fe2d57f7743bb89ab1268...   
4  automl-dm--dpp1-csv-1-ce126b9d82384846b7de42f7...   
5  automl-dm--dpp2-1-a908698600da41f0b73a8110f905...   
6  automl-dm--dpp1-1-c344415cfd204bec8115f5d311cf...   
7  db-1-b0a64fad850445fb80cac729adb2bc6f429367bd4...   

                                         DisplayName  \
0  tuning-job-1-872297ed5107410fbc-001-f28a0f1e-a...   
1  tuning-job-1-872297ed5107410fbc-002-011c5072-a...   
2  tuning-job-1-872297ed5107410fbc-003-6b1c2c2a-a...   
3  automl-dm--dpp2-rpb-1-a57fe2d57f7743bb89ab1268...   
4  automl-dm--dpp1-csv-1-ce126b9d82384846b7de42f7...   
5  automl-dm--dpp2-1-a908698600da41f0b73a8110f905...   
6  automl-dm--dpp1-1-c344415cfd204bec8115f5d311cf...   
7  db-1-b0a64fad850445fb80cac729adb2bc6f429367b

# Explore the Best Candidate
Now that we have successfully completed the AutoML job on our dataset and visualized the trials, we can create a model from any of the trials with a single API call and then deploy that model for online or batch prediction using [Inference Pipelines](https://docs.aws.amazon.com/sagemaker/latest/dg/inference-pipelines.html). For this notebook, we deploy only the best performing trial for inference.

The best candidate is the one we're really interested in.

In [16]:
best_candidate = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)['BestCandidate']
best_candidate_identifier = best_candidate['CandidateName']

print("Candidate name: " + best_candidate_identifier)
print("Metric name: " + best_candidate['FinalAutoMLJobObjectiveMetric']['MetricName'])
print("Metric value: " + str(best_candidate['FinalAutoMLJobObjectiveMetric']['Value']))

Candidate name: tuning-job-1-872297ed5107410fbc-001-f28a0f1e
Metric name: validation:accuracy
Metric value: 0.4808900058269501


In [17]:
best_candidate

{'CandidateName': 'tuning-job-1-872297ed5107410fbc-001-f28a0f1e',
 'FinalAutoMLJobObjectiveMetric': {'MetricName': 'validation:accuracy',
  'Value': 0.4808900058269501},
 'ObjectiveStatus': 'Succeeded',
 'CandidateSteps': [{'CandidateStepType': 'AWS::SageMaker::ProcessingJob',
   'CandidateStepArn': 'arn:aws:sagemaker:us-east-1:835319576252:processing-job/db-1-b0a64fad850445fb80cac729adb2bc6f429367bd427a4d129d64541392',
   'CandidateStepName': 'db-1-b0a64fad850445fb80cac729adb2bc6f429367bd427a4d129d64541392'},
  {'CandidateStepType': 'AWS::SageMaker::TrainingJob',
   'CandidateStepArn': 'arn:aws:sagemaker:us-east-1:835319576252:training-job/automl-dm--dpp2-1-a908698600da41f0b73a8110f905c71685a3d9e6896c4',
   'CandidateStepName': 'automl-dm--dpp2-1-a908698600da41f0b73a8110f905c71685a3d9e6896c4'},
  {'CandidateStepType': 'AWS::SageMaker::TransformJob',
   'CandidateStepArn': 'arn:aws:sagemaker:us-east-1:835319576252:transform-job/automl-dm--dpp2-rpb-1-a57fe2d57f7743bb89ab1268334871f1e4ab

We can see the containers and models composing the Inference Pipeline.

In [18]:
for container in best_candidate['InferenceContainers']:
    print(container['Image'])
    print(container['ModelDataUrl'])
    print('======================')

683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-sklearn-automl:0.1.0-cpu-py3
s3://sagemaker-us-east-1-835319576252/models/autopilot/automl-dm-16-05-20-45/data-processor-models/automl-dm--dpp2-1-a908698600da41f0b73a8110f905c71685a3d9e6896c4/output/model.tar.gz
683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:0.90-1-cpu-py3
s3://sagemaker-us-east-1-835319576252/models/autopilot/automl-dm-16-05-20-45/tuning/automl-dm--dpp2-xgb/tuning-job-1-872297ed5107410fbc-001-f28a0f1e/output/model.tar.gz
683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-sklearn-automl:0.1.0-cpu-py3
s3://sagemaker-us-east-1-835319576252/models/autopilot/automl-dm-16-05-20-45/data-processor-models/automl-dm--dpp2-1-a908698600da41f0b73a8110f905c71685a3d9e6896c4/output/model.tar.gz


# AutoPilot Chooses XGBoost as Best Candidate!
Note that AutoPilot chose different hyper-parameters and feature transformations than we used in our own XGBoost model.

# Deploy the Model as a REST Endpoint
Batch transformations are also supported, but for now, we will use a REST Endpoint.

In [19]:
model_name = 'automl-dm-model-' + timestamp_suffix

model_arn = sm.create_model(Containers=best_candidate['InferenceContainers'],
                            ModelName=model_name,
                            ExecutionRoleArn=role)

print('Best candidate model ARN: ', model_arn['ModelArn'])

Best candidate model ARN:  arn:aws:sagemaker:us-east-1:835319576252:model/automl-dm-model-16-05-20-45


In [20]:
# EndpointConfig name
timestamp_suffix = strftime('%d-%H-%M-%S', gmtime())
epc_name = 'automl-dm-epc-' + timestamp_suffix

# Endpoint name
xgb_endpoint_name = 'automl-dm-ep-' + timestamp_suffix
variant_name = 'automl-dm-variant-' + timestamp_suffix

print(xgb_endpoint_name)
print(variant_name)

automl-dm-ep-16-05-42-55
automl-dm-variant-16-05-42-55


In [21]:
ep_config = sm.create_endpoint_config(EndpointConfigName = epc_name,
                                      ProductionVariants=[{'InstanceType':'ml.m4.xlarge',
                                                           'InitialInstanceCount': 1,
                                                           'ModelName': model_name,
                                                           'VariantName': variant_name}])


In [22]:
create_endpoint_response = sm.create_endpoint(EndpointName=xgb_endpoint_name,
                                              EndpointConfigName=epc_name)
print(create_endpoint_response['EndpointArn'])

arn:aws:sagemaker:us-east-1:835319576252:endpoint/automl-dm-ep-16-05-42-55


# Wait for the Model to Deploy
This may take 5-10 mins.  Please be patient.

In [23]:
sm.get_waiter('endpoint_in_service').wait(EndpointName=xgb_endpoint_name)


In [24]:
resp = sm.describe_endpoint(EndpointName=xgb_endpoint_name)
status = resp['EndpointStatus']

print("Arn: " + resp['EndpointArn'])
print("Status: " + status)

Arn: arn:aws:sagemaker:us-east-1:835319576252:endpoint/automl-dm-ep-16-05-42-55
Status: InService


# Test Our Model with Some Example Reviews
Let's do some ad-hoc predictions on our model.

In [25]:
sm_runtime = boto3.client('sagemaker-runtime')

In [26]:
csv_line_predict_positive = """I loved it!"""

response = sm_runtime.invoke_endpoint(EndpointName=xgb_endpoint_name, ContentType='text/csv', Accept='text/csv', Body=csv_line_predict_positive)

response_body = response['Body'].read().decode('utf-8').strip()
response_body

'5'

In [28]:
csv_line_predict_meh = """It's OK."""

response = sm_runtime.invoke_endpoint(EndpointName=xgb_endpoint_name, ContentType='text/csv', Accept='text/csv', Body=csv_line_predict_meh)

response_body = response['Body'].read().decode('utf-8').strip()
response_body

'3'

In [27]:
csv_line_predict_negative = """The worst product ever."""

response = sm_runtime.invoke_endpoint(EndpointName=xgb_endpoint_name, ContentType='text/csv', Accept='text/csv', Body=csv_line_predict_negative)

response_body = response['Body'].read().decode('utf-8').strip()
response_body

'1'

# Summary
We used AutoPilot to automatically find the best model, hyper-parameters, and feature-engineering scripts for our dataset.  

AutoPilot uses a white-box approach to generate re-usable exploration Jupyter Notebooks and transformation Python scripts to continue to train and deploy our model on new data - well after this initial interaction with the AutoPilot service.